In [44]:
import praw
from psaw import PushshiftAPI
import datetime as dt
import pandas as pd
import shutil
import os
from tqdm.notebook import tqdm_notebook
import json
import time

In [5]:
REDDIT_CLIENT_ID = '4FEO91ch16g-P_n8AxU2_A'
REDDIT_CLIENT_SECRET = 'Q2cyR53G5a6IpsSAzLbevNtBiITYhw'
REDDIT_USER_AGENT = 'desktop:DSE203 (by u/Life_is_Life)'

SUBREDDITS_TO_EXTRACT = (
    # Sources:
    #  - https://thehiveindex.com/topics/investing/platform/reddit/
    #  - https://www.investopedia.com/reddit-top-investing-and-trading-communities-5189322
    'stocks',
    'wallstreetbets',
    'pennystocks',
    'investing',
    'Wallstreetbetsnew',
    'StockMarket',
    'options',
    'RobinHood',
    'RobinHoodPennyStocks',
    'weedstocks',
    'smallstreetbets',
    'SecurityAnalysis',
    'CanadianInvestor',
    'SPACs',
    'InvestmentClub',
    'ValueInvesting',
    'investing_discussion',
    'stonks',
    'shroomstocks',
)

ROOT_SAVE_DIR = 'top_reddit_posts_and_comments'
JOB_DIR = '1638719861'

In [64]:
reddit = praw.Reddit(
    client_id = REDDIT_CLIENT_ID,
    client_secret = REDDIT_CLIENT_SECRET,
    user_agent = REDDIT_USER_AGENT
)

api = PushshiftAPI()

In [11]:
if not os.path.exists(os.path.join(ROOT_SAVE_DIR, JOB_DIR)):
    raise FileNotFoundError()
    
os.makedirs(os.path.join(ROOT_SAVE_DIR, 'details'), exist_ok=False)

In [32]:
reddit.config.ratelimit_seconds = 1

reddit.config.ratelimit_seconds

1

In [45]:
post_id_file = '20211125.json'

date_data_list = list()
for d in tqdm_notebook(json.load(open(os.path.join(ROOT_SAVE_DIR, JOB_DIR, post_id_file)))):
    reddit_submission = reddit.submission(id=d['id'])
    
    submission_url = reddit_submission.url
    submission_title = reddit_submission.title
    submission_selftext = reddit_submission.selftext
    submission_created = dt.datetime.fromtimestamp(reddit_submission.created).isoformat()

    submission_comments = list()
    for submission_comment in reddit_submission.comments:
        if isinstance(submission_comment, praw.models.MoreComments):
            continue

        submission_comments.append({
            'author': submission_comment.author.name if submission_comment.author is not None else None,
            'created': dt.datetime.fromtimestamp(submission_comment.created).isoformat(),
            'body': submission_comment.body,
        })
    
    date_data_list.append({
        'id': d['id'],
        'url': submission_url,
        'title': submission_title,
        'selftext': submission_selftext,
        'created': submission_created,
        'comments': submission_comments,
    })
    
    time.sleep(0.5)
    
with open(os.path.join(ROOT_SAVE_DIR, 'details', post_id_file), 'w') as f:
    json.dump(date_data_list, f, indent=4)

  0%|          | 0/297 [00:00<?, ?it/s]

AttributeError: 'MoreComments' object has no attribute 'author'

In [74]:
with tqdm_notebook(total=len(SUBREDDITS_TO_EXTRACT) * len(dates_to_extract)) as pbar:
    for (start_epoch, end_epoch) in dates_to_extract[::-1]:
        date_data_list = list()
        
        for subreddit_to_extract in SUBREDDITS_TO_EXTRACT:
            submissions = list(api.search_submissions(
                after = start_epoch,
                before = end_epoch,
                subreddit = subreddit_to_extract,
                sort_type = 'num_comments',
                filters = ['id', 'score', 'num_comments'],
                sort = 'desc',
                limit = 25
            ))
            
            for submission in submissions:
                submission_id = submission.id
            
                date_data_list.append({
                    'subreddit': subreddit_to_extract,
                    'id': submission_id,
                    'score': submission.score,
                    'num_comments': submission.num_comments,
                })
            
            # Update progress bar
            pbar.update()

  0%|          | 0/38 [00:00<?, ?it/s]

In [59]:
START_DATE = '2021-01-01'
END_DATE = '2021-01-02'

# This object is a list. Each item in the list will have a 2-tuple (start_epoch, end_epoch)
dates_to_extract = list()

for date_to_extract in pd.date_range(START_DATE, END_DATE):
    dates_to_extract.append(
        tuple([
            int(date_to_extract.to_pydatetime().timestamp()),
            int((date_to_extract + pd.Timedelta(days=1) - pd.Timedelta(seconds=1)).to_pydatetime().timestamp())
        ])
    )

print(len(dates_to_extract))
print(dates_to_extract[0])

2
(1609488000, 1609574399)


In [ ]:
d['id']

'r2fyun'

In [ ]:
mys = reddit.submission(id=d['id'])

mys

Submission(id='r2fyun')